## 🧠💡 Intelligent Systems  for Smart Health 👨‍⚕👩‍⚕️🔬🌡️


# Finding hyperparameters and understanding models

In the last weeks we trained linear models, decision tree and a random forest. In all those cases we were manually adjusting hyperparameters. The more complex our models get... the more inefficient this procedure becomes. So we will look at **grid search** and **random search** as two common techniques for (still fairly simple) hyperparameter searches.

In the second part of this session we will then ask the question of how to make sense of the predictions we get from such an optimized machine learning model. Why does one patient get a good prognosis, and another one a bad one?
In this context, we will look at the **feature importance** (for a random forest model) and and **SHAP**, a newer technique to interpret model predictions.

We will (again) work with actual medical data in this notebook, namely the NHANES I epidemiology dataset (for a detailed description of this dataset you can check the [CDC Website](https://wwwn.cdc.gov/nchs/nhanes/nhefs/default.aspx/)).


<a name='import'></a>
## Import Packages

We'll first import all the common packages that we need for this assignment. 

- `shap` is a library that explains predictions made by machine learning models.
- `sklearn` is one of the most popular machine learning libraries.
- `itertools` allows us to conveniently manipulate iterable objects such as lists.
- `pydotplus` is used together with `IPython.display.Image` to visualize graph structures such as decision trees.
- `numpy` is a fundamental package for scientific computing in Python.
- `pandas` is what we'll use to manipulate our data.
- `seaborn` is a plotting library which has some convenient functions for visualizing missing data.
- `matplotlib` is a plotting library.

In [ ]:
#!pip install pydotplus
#!pip install lifelines
#!pip install shap

In [ ]:
import os
import shap
import sklearn
import itertools
import pydotplus
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Image 

from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

<a name='1'></a>
## 1. The Dataset
### Load and explore the data!

In virtually all cases, we would first want to get an intuition on the data itself. Things like: What is in the data? How much data is there? Are there things missing? What might cause problems? Do we understand the type of data/features?

With **pandas**, we usually can explore some key properties very rapidly, for instance with commands like

- `data.head()`
- `data.describe()`
- `data.info()`

### Some weird conventions:
For some reason it became standard in the machine learning world to name the actual data `X` and the labels `y`. Even though I consider this a rather poor choice both from a math and from a code best practice point of view, we will stick to this in this notebook.

In [ ]:
path_data = "../data/"

X = pd.read_csv(os.path.join(path_data, "NHANESI_subset_X.csv"))
y = pd.read_csv(os.path.join(path_data, "NHANESI_subset_y.csv"))

X = X.drop('Unnamed: 0', axis=1)
y = y.drop('Unnamed: 0', axis=1)

X.head()

In [ ]:
data = X.copy()
data["time"] = y
data = data.dropna(axis="rows")
data.head()

In [ ]:
period = 10  # time period we consider --> we focus on >10 year risk

def died_in_period(time, period):
    # Option 1: Person died within a period
    if time > 0 and time < period:
        return 1

    # Option 2: Person left study before period ended --> no conclusion possible
    if time <= 0 and time > -period:
        return np.nan

    # Option 3 + 4: Person left study after >= period or survived >= period
    return 0
    
death_in_period = np.array([died_in_period(time, period) for time in data.time])
death_in_period

In [ ]:
data["death_in_period"] = death_in_period

# Remove missing entries
data = data.dropna()

In [ ]:
data.death_in_period.value_counts()

In [ ]:
# Create data/label split --> X, y
X = data.drop(["death_in_period", "time"], axis='columns')
y = data["death_in_period"]

In [ ]:
X.shape, y.shape

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=10
)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_dev, y_dev,
    test_size=0.18,
    random_state=10
)

In [ ]:
X_train.shape, X_val.shape, X_test.shape

In [ ]:
# in this context we will also use the c-index to evaluate our models
import lifelines

def cindex(y_true, scores):
    return lifelines.utils.concordance_index(y_true, scores)

<a name='2'></a>
## 2. Random Forests

As we saw last time, a single decision tree is prone to overfitting. To solve this problem, you can try **random forests**, which combine predictions from many different trees to create a robust classifier. 

As before, we will use scikit-learn to build a random forest for the data. We will use the default hyperparameters.

Using Scikit-Learn we can train such a model using the `RandomForestClassifier()` class and, again, the `fit()` method to train the model.

In [ ]:
# please train a random forest model using the RandomForestClassifier() class
# start with 10 trees --> n_estimators=10
forest = 

### Basic first model evaluation
Here, for a start, we will simply use the C-Index. Feel free to add other measures (e.g., accuracy etc.).

In [ ]:
y_train_preds = forest.predict_proba(X_train)[:, 1]
print(f"Train C-Index: {cindex(y_train.values, y_train_preds)}")

y_val_preds = forest.predict_proba(X_val)[:, 1]
print(f"Val C-Index: {cindex(y_val.values, y_val_preds)}")

Training a random forest with the default hyperparameters results in a model that has better predictive performance than individual decision trees as in the previous section, but this model is overfitting.

We therefore need to tune (or optimize) the hyperparameters, to find a model that both has good predictive performance and minimizes overfitting.

The hyperparameters we choose to adjust will be:

- `n_estimators`: the number of trees used in the forest.
- `max_depth`: the maximum depth of each tree.
- `min_samples_leaf`: the minimum number (if `int`) or proportion (if `float`) of samples in a leaf.


<a name='ex1'></a>
### Exercise 1: try to find a set of better hyperparameters!


In [ ]:
# train some models using different hyperparameters
forest = # add your code

In [ ]:
y_train_preds = forest.predict_proba(X_train)[:, 1]
print(f"Train C-Index: {cindex(y_train.values, y_train_preds)}")

y_val_preds = forest.predict_proba(X_val)[:, 1]
print(f"Val C-Index: {cindex(y_val.values, y_val_preds)}")

<a name='3'></a>
## 3.Systematic search for hyperparameters: grid search

The approach we implement to tune the hyperparameters is known as a grid search:

- We define a set of possible values for each of the target hyperparameters.

- A model is trained and evaluated for every possible combination of hyperparameters.

- The best performing set of hyperparameters is returned.

The cell below implements a hyperparameter grid search, using the C-Index to evaluate each tested model.

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {"max_depth": ...
             }


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [4, 8, 12, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'n_estimators': [2, 10, 100]},
             verbose=2)

In [ ]:
# Get the best model out of the grid search with .best_params_


In [ ]:
forest = # add code to use the best model

y_train_preds = forest.predict_proba(X_train)[:, 1]
print(f"Train C-Index: {cindex(y_train.values, y_train_preds)}")

y_val_preds = forest.predict_proba(X_val)[:, 1]
print(f"Val C-Index: {cindex(y_val.values, y_val_preds)}")

y_test_preds = forest.predict_proba(X_test)[:, 1]
print(f"Test C-Index: {cindex(y_test.values, y_test_preds)}")

In [ ]:
# Collect grid search results
cv_results = grid_search.cv_results_
mean_test_scores = cv_results['mean_test_score']
params = cv_results['params']

# Prepare data for plotting
scores_array = np.array(mean_test_scores).reshape(len(parameters['max_depth']),
                                                  len(parameters['min_samples_leaf']),
                                                  len(parameters['n_estimators']))

# Create a line plot for each n_estimators
fig, ax = plt.subplots(figsize=(10, 6))

for k, n_estimators in enumerate(parameters['n_estimators']):
    scores_for_n_estimators = scores_array[:, :, k].T
    for i, min_samples_leaf in enumerate(parameters['min_samples_leaf']):
        ax.plot(parameters['max_depth'], scores_for_n_estimators[i],
                marker='o', linestyle='--', label=f'n_estimators: {n_estimators}, min_samples_leaf: {min_samples_leaf}')

ax.set_title('Grid Search Results')
ax.set_xlabel('Max Depth')
ax.set_ylabel('Mean Test Score')
ax.legend(loc='best')

plt.show()

## Random Search
A common alternative to the simple grid search is a more randomized search. The version that is implemented in Scikit-learn is a bit of a hybrid. It can be seen as a randomized search within a grid.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {"max_depth": [],
              "min_samples_leaf": [],
              "n_estimators": []
             }

forest = RandomForestClassifier()
random_search = # add your code

In [ ]:
xlabel = "n_estimators"
ylabel = "max_depth"
zlabel = "min_samples_leaf"
x_pos = []
y_pos = []
z_pos = []
for param in random_search.cv_results_["params"]:
    x_pos.append(param[xlabel])
    y_pos.append(param[ylabel])
    z_pos.append(param[zlabel])

plt.scatter(x_pos, y_pos, s=50*np.array(z_pos),
            c=random_search.cv_results_["mean_test_score"])
plt.colorbar(label="mean test score")
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.xscale("log")

In [ ]:
# take the "best" model
model = # add your code

## 4. Optional (if time) Training a lightGBM model

XGBoost models became somewhat famous on platforms such as Kaggle, because they were very often found in the top ranks of the leadboards. So let's give this a try as well!

https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

<a name='5'></a>
## 5. Explainability
Using a random forest has improved results, but we've lost some of the natural interpretability of trees. In this section we'll try to explain the predictions using slightly more sophisticated techniques. 

But first, we will simply look at the **feature importance**:

---
### Just to all be on the same page: let's train one last model!

In [ ]:
model = RandomForestClassifier(n_estimators=100,
                               max_depth=7,
                               min_samples_leaf=5,
                               random_state=10)

model.fit(X_train, y_train)

In [ ]:
# use .feature_importances_ to get the respective values of our model

In [ ]:
feature_importance = pd.DataFrame(# add feature importances here,
                                 columns=["feature_importance"],
                                 index=X_train.columns)
feature_importance.sort_values("feature_importance", ascending=False)

<a name='shap'></a>
## Better than only feature importance: SHAP

Looking at the feature importance was interesting, and tells us a bit more about the model.
But since recently, we can do much better!

Now we will apply **SHAP (SHapley Additive exPlanations)**, a cutting edge method that explains predictions made by black-box machine learning models (i.e. models which are too complex to be understandable by humans as is).

> Given a prediction made by a machine learning model, SHAP values explain the prediction by quantifying the additive importance of each feature to the prediction. SHAP values have their roots in cooperative game theory, where Shapley values are used to quantify the contribution of each player to the game.
> 
> Although it is computationally expensive to compute SHAP values for general black-box models, in the case of trees and forests there exists a fast polynomial-time algorithm. For more details, see the [TreeShap paper](https://arxiv.org/pdf/1802.03888.pdf).

We'll use the [shap library](https://github.com/slundberg/shap) to do this for our random forest model. Run the next cell to output the most at risk individuals in the test set according to our model.

In [ ]:
proba_death = model.predict_proba(X_test)[:, 1]

In [ ]:
X_test_risk = X_test.copy(deep=True)
X_test_risk["predicted_risk"] = proba_death
X_test_risk = X_test_risk.sort_values("predicted_risk", ascending=False)
X_test_risk.head()

We can use SHAP values to try and understand the model output on specific individuals using force plots. Run the cell below to see a force plot on the riskiest individual. 

In [ ]:
i = 100
patientID = X_test_risk.index[i]
print(patientID)
print(X_test.loc[patientID, :], "\n")
print(f"Our model predicts: {model.predict(X_test.loc[[patientID]])}")

### Why this prediction?

### Using SHAP's general "Explainer"

In [ ]:
explainer = # add your code

In [ ]:
shap_values = # add your code

In [ ]:
prediction_class = 1

shap.force_plot(
    explainer.expected_value[prediction_class], shap_values[:, prediction_class],
    feature_names=X_test.columns,
    matplotlib=True
)

### Using SHAP's "TreeExplainer"

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
shap_values = # add your code

In [ ]:
shap.force_plot(
    explainer.expected_value[prediction_class], shap_value,
    feature_names=X_test.columns,
    matplotlib=True
)

### Summary plot

In [ ]:
explainer = shap.Explainer(...)
shap_values = explainer(..., check_additivity=False)

In [ ]:
shap.plots.beeswarm(shap_values[:,:,0])

In [ ]:
# or

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(...)

shap.summary_plot(...)